找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。


In [1]:
import _thread
import time
import random

from bs4 import BeautifulSoup
import requests

原本的爬蟲時間

In [2]:
proxy_ips = []

url = 'https://free-proxy-list.net/'
r = requests.get(url)
soup = BeautifulSoup(r.text)

table = soup.find('tbody')

for row in table:
    td = row.find_all('td')
    ip = td[0].text
    port = td[1].text
    proxy_ips.append(f'http://{ip}:{port}')

In [3]:
def try_proxy(proxy_ip):
    try:
        resp = requests.get('http://ip.filefab.com/index.php',
                        proxies={'http': ip}, timeout=10)
        soup = BeautifulSoup(resp.text, 'lxml')
        
        print(soup.find('h1', id='ipd').text.strip())

    except Exception as e:
        print('Fail: ', e)

In [4]:
startTime = time.time()
for ip in proxy_ips[:10]:
    print('Use', ip)
    try_proxy(ip)

finishTime = time.time()
print('花費時間: ', (finishTime - startTime)) # 正常情況的爬蟲所需時間

Use http://186.42.186.202:48281
Fail:  HTTPConnectionPool(host='186.42.186.202', port=48281): Max retries exceeded with url: http://ip.filefab.com/index.php (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000002978897D188>, 'Connection to 186.42.186.202 timed out. (connect timeout=10)'))
Use http://117.102.94.148:34742
Your IP address: 117.102.94.148
Use http://104.43.244.233:80
Fail:  HTTPConnectionPool(host='104.43.244.233', port=80): Max retries exceeded with url: http://ip.filefab.com/index.php (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000002978897F748>, 'Connection to 104.43.244.233 timed out. (connect timeout=10)'))
Use http://23.91.219.238:8080
Fail:  HTTPConnectionPool(host='23.91.219.238', port=8080): Max retries exceeded with url: http://ip.filefab.com/index.php (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000297889A1D48>, 'Connection to 23.91.219.238 timed out. (connect timeo

加入多執行緒

In [5]:
startTime = time.time()
threads = []
for i in proxy_ips[:10]:
    print('Use', i)
    _thread.start_new_thread(try_proxy, (i, ))

finishTime = time.time()
print('花費時間: ', (finishTime - startTime))

Use http://186.42.186.202:48281
Use http://117.102.94.148:34742
Use http://104.43.244.233:80
Use http://23.91.219.238:8080
Use http://208.98.185.89:53630
Use http://159.89.40.245:8118
Use http://154.127.120.18:30280
Use http://185.162.142.81:53281
Use http://154.119.79.254:53281
Use http://118.174.232.237:48665
花費時間:  0.03803873062133789
